In [17]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


# Using TOT Sector Datasets

# Datasets : Residential Sector

In [18]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_excel('Datasets/use_tot_sector.xlsx' , sheet_name='Residential Sector')


In [19]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,10189,10787,11546,12335,14082,15020,16611,17607,19393,...,53939,54597,48810,47823,53136,50844,53086,49404,48533,54191
1,AL,124894,121253,132329,140993,148042,147307,153849,155314,187648,...,377607,338290,358238,378853,349230,341584,323378,358163,347877,331510
2,AR,80587,80624,82565,85776,93756,90025,110858,113653,124499,...,247420,222034,241000,246790,228572,213798,202040,235035,226891,216401
3,AZ,48824,51198,48300,52660,60457,58339,63080,66568,72096,...,391384,378865,389450,377957,385470,392068,401347,407307,412796,441556
4,CA,595971,619172,677892,711654,792464,841483,870428,932522,948310,...,1500559,1437024,1440463,1355028,1347770,1367103,1415455,1440273,1455751,1507721


# Data statistics

In [20]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,10189,10787,11546,12335,14082,15020,16611,17607,19393,...,53939,54597,48810,47823,53136,50844,53086,49404,48533,54191
1,AL,124894,121253,132329,140993,148042,147307,153849,155314,187648,...,377607,338290,358238,378853,349230,341584,323378,358163,347877,331510
2,AR,80587,80624,82565,85776,93756,90025,110858,113653,124499,...,247420,222034,241000,246790,228572,213798,202040,235035,226891,216401
3,AZ,48824,51198,48300,52660,60457,58339,63080,66568,72096,...,391384,378865,389450,377957,385470,392068,401347,407307,412796,441556
4,CA,595971,619172,677892,711654,792464,841483,870428,932522,948310,...,1500559,1437024,1440463,1355028,1347770,1367103,1415455,1440273,1455751,1507721


In [21]:
df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,...,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01
mean,3.474304e+05,3.569242e+05,3.760358e+05,3.839395e+05,3.935676e+05,4.089378e+05,4.291058e+05,4.473644e+05,4.742475e+05,5.062079e+05,...,8.209836e+05,7.626392e+05,8.083886e+05,8.237700e+05,7.917877e+05,7.752942e+05,7.639132e+05,8.264203e+05,8.093101e+05,7.892537e+05
std,1.241968e+06,1.275971e+06,1.344443e+06,1.372569e+06,1.406983e+06,1.462480e+06,1.534342e+06,1.599589e+06,1.694738e+06,1.808675e+06,...,2.927031e+06,2.719264e+06,2.881524e+06,2.936140e+06,2.823072e+06,2.764511e+06,2.724045e+06,2.946493e+06,2.885534e+06,2.814985e+06
min,7.144000e+03,7.337000e+03,7.736000e+03,8.699000e+03,9.378000e+03,9.876000e+03,1.050000e+04,1.172100e+04,1.263800e+04,1.405600e+04,...,3.250600e+04,3.153000e+04,3.084800e+04,3.140700e+04,3.157100e+04,3.250600e+04,3.402000e+04,3.423700e+04,3.454300e+04,3.662700e+04
25%,4.289450e+04,4.631475e+04,4.829850e+04,5.119550e+04,5.025425e+04,4.901650e+04,5.374550e+04,5.693975e+04,5.948750e+04,6.316175e+04,...,1.241072e+05,1.163580e+05,1.259738e+05,1.190602e+05,1.215718e+05,1.216165e+05,1.287710e+05,1.273908e+05,1.358190e+05,1.317632e+05
50%,1.194265e+05,1.199260e+05,1.288600e+05,1.345520e+05,1.400340e+05,1.415940e+05,1.472070e+05,1.511680e+05,1.704870e+05,1.826450e+05,...,3.621665e+05,3.366575e+05,3.471980e+05,3.508540e+05,3.404320e+05,3.291425e+05,3.179110e+05,3.464650e+05,3.330415e+05,3.161910e+05
75%,2.334032e+05,2.378325e+05,2.460572e+05,2.457312e+05,2.461472e+05,2.540045e+05,2.685550e+05,2.754682e+05,2.946808e+05,3.167120e+05,...,5.659315e+05,5.248722e+05,5.722485e+05,5.857522e+05,5.468952e+05,5.325250e+05,5.118898e+05,5.688348e+05,5.431800e+05,5.234192e+05
max,9.033190e+06,9.280028e+06,9.776931e+06,9.982426e+06,1.023276e+07,1.063238e+07,1.115675e+07,1.163148e+07,1.233044e+07,1.316140e+07,...,2.134557e+07,1.982862e+07,2.101810e+07,2.141802e+07,2.058648e+07,2.015765e+07,1.986174e+07,2.148693e+07,2.104206e+07,2.052060e+07


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   52 non-null     object
 1   1960    52 non-null     int64 
 2   1961    52 non-null     int64 
 3   1962    52 non-null     int64 
 4   1963    52 non-null     int64 
 5   1964    52 non-null     int64 
 6   1965    52 non-null     int64 
 7   1966    52 non-null     int64 
 8   1967    52 non-null     int64 
 9   1968    52 non-null     int64 
 10  1969    52 non-null     int64 
 11  1970    52 non-null     int64 
 12  1971    52 non-null     int64 
 13  1972    52 non-null     int64 
 14  1973    52 non-null     int64 
 15  1974    52 non-null     int64 
 16  1975    52 non-null     int64 
 17  1976    52 non-null     int64 
 18  1977    52 non-null     int64 
 19  1978    52 non-null     int64 
 20  1979    52 non-null     int64 
 21  1980    52 non-null     int64 
 22  1981    52 non-null     int6

In [23]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,10189,10787,11546,12335,14082,15020,16611,17607,19393,...,53939,54597,48810,47823,53136,50844,53086,49404,48533,54191
1,AL,124894,121253,132329,140993,148042,147307,153849,155314,187648,...,377607,338290,358238,378853,349230,341584,323378,358163,347877,331510
2,AR,80587,80624,82565,85776,93756,90025,110858,113653,124499,...,247420,222034,241000,246790,228572,213798,202040,235035,226891,216401
3,AZ,48824,51198,48300,52660,60457,58339,63080,66568,72096,...,391384,378865,389450,377957,385470,392068,401347,407307,412796,441556
4,CA,595971,619172,677892,711654,792464,841483,870428,932522,948310,...,1500559,1437024,1440463,1355028,1347770,1367103,1415455,1440273,1455751,1507721


# Transformation of Data

In [24]:
df_trans = df.melt(id_vars=['State'] , var_name='Year', value_name='Yearly Data')
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans


,State,Year,Yearly Data
0,AK,1960-01-01,10189
1,AL,1960-01-01,124894
2,AR,1960-01-01,80587
3,AZ,1960-01-01,48824
4,CA,1960-01-01,595971
...,...,...,...
3167,WA,2020-01-01,464617
3168,WI,2020-01-01,428482
3169,WV,2020-01-01,151239
3170,WY,2020-01-01,52008


# Modeling the Data
## Using ARIMA Model

In [25]:
os.makedirs('Plots/use_tot_Sector_data/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Residential Sector : Energy Consumption Forecast State : {State} using ARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_tot_Sector_data/Arima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        

Mean Absolute Error (MAE): 4266.96
Mean Squared Error (MSE): 23351251.52
Mean Absolute Percentage Error (MAPE): 8.51


Mean Absolute Error (MAE): 28591.19
Mean Squared Error (MSE): 997559407.92
Mean Absolute Percentage Error (MAPE): 8.53


Mean Absolute Error (MAE): 22838.31
Mean Squared Error (MSE): 630568886.01
Mean Absolute Percentage Error (MAPE): 10.69


Mean Absolute Error (MAE): 7182.29
Mean Squared Error (MSE): 136583823.86
Mean Absolute Percentage Error (MAPE): 1.67


Mean Absolute Error (MAE): 137070.12
Mean Squared Error (MSE): 23668829953.82
Mean Absolute Percentage Error (MAPE): 9.39


...

Mean Absolute Error (MAE): 17685.15
Mean Squared Error (MSE): 434300160.31
Mean Absolute Percentage Error (MAPE): 3.63


Mean Absolute Error (MAE): 15623.2
Mean Squared Error (MSE): 379823246.8
Mean Absolute Percentage Error (MAPE): 3.55


Mean Absolute Error (MAE): 11176.62
Mean Squared Error (MSE): 169539891.58
Mean Absolute Percentage Error (MAPE): 7.45


Mean Absolute Error (MAE): 1912.64
Mean Squared Error (MSE): 4793057.83
Mean Absolute Percentage Error (MAPE): 3.66


Mean Absolute Error (MAE): 710971.51
Mean Squared Error (MSE): 637150352041.82
Mean Absolute Percentage Error (MAPE): 3.49


## Using SARIMA Model

In [26]:
os.makedirs('Plots/use_tot_Sector_data/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Residential Sector : Energy Consumption Forecast State : {State} using SARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_tot_Sector_data/Sarima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        # break

Mean Absolute Error (MAE): 4266.96
Mean Squared Error (MSE): 23351251.52
Mean Absolute Percentage Error (MAPE): 8.51


Mean Absolute Error (MAE): 28591.19
Mean Squared Error (MSE): 997559407.92
Mean Absolute Percentage Error (MAPE): 8.53


Mean Absolute Error (MAE): 22838.31
Mean Squared Error (MSE): 630568886.01
Mean Absolute Percentage Error (MAPE): 10.69


Mean Absolute Error (MAE): 7182.29
Mean Squared Error (MSE): 136583823.86
Mean Absolute Percentage Error (MAPE): 1.67


Mean Absolute Error (MAE): 137070.12
Mean Squared Error (MSE): 23668829953.82
Mean Absolute Percentage Error (MAPE): 9.39


...

Mean Absolute Error (MAE): 17685.15
Mean Squared Error (MSE): 434300160.31
Mean Absolute Percentage Error (MAPE): 3.63


Mean Absolute Error (MAE): 15623.2
Mean Squared Error (MSE): 379823246.8
Mean Absolute Percentage Error (MAPE): 3.55


Mean Absolute Error (MAE): 11176.62
Mean Squared Error (MSE): 169539891.58
Mean Absolute Percentage Error (MAPE): 7.45


Mean Absolute Error (MAE): 1912.64
Mean Squared Error (MSE): 4793057.83
Mean Absolute Percentage Error (MAPE): 3.66


Mean Absolute Error (MAE): 710971.51
Mean Squared Error (MSE): 637150352041.82
Mean Absolute Percentage Error (MAPE): 3.49
